In [18]:
import numpy as np
import pandas as pd
import scipy.io as sio
import neurokit2 as nk
from sklearn.model_selection import GroupKFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
import joblib
import time
from os.path import exists
import matplotlib.pyplot as plt

In [19]:
def feat_extract_ECG(raw):
    data_ECG = {}
    for participant in range(0, 23):
        for video in range(0, 18):

            stim_left = (raw["DREAMER"][0, 0]["Data"]
                         [0, participant]["ECG"][0, 0]
                         ["stimuli"][0, 0][video, 0][:, 0])
            stim_right = (raw["DREAMER"][0, 0]["Data"]
                          [0, participant]["ECG"][0, 0]
                          ["stimuli"][0, 0][video, 0][:, 1])

            signals_s_l, info_s_l = nk.ecg_process(
                stim_left, sampling_rate=256)
            signals_s_r, info_s_r = nk.ecg_process(
                stim_right, sampling_rate=256)

            features_ecg_l = nk.ecg_intervalrelated(signals_s_l)
            features_ecg_r = nk.ecg_intervalrelated(signals_s_r)

            features_ecg = (features_ecg_l + features_ecg_r)/2
            if not len(data_ECG):
                data_ECG = features_ecg
            else:
                data_ECG = pd.concat([data_ECG, features_ecg],
                                     ignore_index=True)
    return data_ECG

In [20]:
def participant_affective(raw):
    a = np.zeros((23, 18, 9), dtype=object)
    for participant in range(0, 23):
        for video in range(0, 18):
            a[participant, video, 0] = (raw["DREAMER"][0, 0]["Data"]
                                        [0, participant]["Age"][0][0][0])
            a[participant, video, 1] = (raw["DREAMER"][0, 0]["Data"]
                                        [0, participant]["Gender"][0][0][0])
            a[participant, video, 2] = int(participant+1)
            a[participant, video, 3] = int(video+1)
            a[participant, video, 4] = ["Searching for Bobby Fischer",
                                        "D.O.A.", "The Hangover", "The Ring",
                                        "300", "National Lampoon\'s VanWilder",
                                        "Wall-E", "Crash", "My Girl",
                                        "The Fly", "Pride and Prejudice",
                                        "Modern Times", "Remember the Titans",
                                        "Gentlemans Agreement", "Psycho",
                                        "The Bourne Identitiy",
                                        "The Shawshank Redemption",
                                        "The Departed"][video]
            a[participant, video, 5] = ["calmness", "surprise", "amusement",
                                        "fear", "excitement", "disgust",
                                        "happiness", "anger", "sadness",
                                        "disgust", "calmness", "amusement",
                                        "happiness", "anger", "fear",
                                        "excitement", "sadness",
                                        "surprise"][video]
            a[participant, video, 6] = int(raw["DREAMER"][0, 0]["Data"]
                                           [0, participant]["ScoreValence"]
                                           [0, 0][video, 0])
            a[participant, video, 7] = int(raw["DREAMER"][0, 0]["Data"]
                                           [0, participant]["ScoreArousal"]
                                           [0, 0][video, 0])
            a[participant, video, 8] = int(raw["DREAMER"][0, 0]["Data"]
                                           [0, participant]["ScoreDominance"]
                                           [0, 0][video, 0])
    b = pd.DataFrame(a.reshape((23*18, a.shape[2])),
                     columns=["age", "gender", "participant",
                              "video", "video_name", "target_emotion",
                              "valence", "arousal", "dominance"])
    return b


In [21]:
def customiz_data_ML(data):
    data = pd.DataFrame(data, columns=[
        'ECG_Rate_Mean', 'HRV_MeanNN', 'HRV_SDNN', 'HRV_RMSSD', 'HRV_SDSD', 'HRV_CVNN',
        'HRV_CVSD', 'HRV_MedianNN', 'HRV_MadNN', 'HRV_MCVNN', 'HRV_IQRNN', 'HRV_HTI', 
        'HRV_HF', 'HRV_HFn', 'HRV_LnHF', 'HRV_SD1', 'HRV_SD2', 'HRV_SD1SD2', 'HRV_S', 
        'HRV_CSI', 'HRV_CVI', 'HRV_CSI_Modified', 'HRV_PIP', 'HRV_IALS', 'HRV_PSS', 
        'HRV_GI', 'HRV_SI', 'HRV_AI', 'HRV_PI', 'HRV_C1d', 'HRV_C1a', 'HRV_SD1d', 'HRV_SD1a',
        'HRV_C2d', 'HRV_C2a', 'HRV_SD2d', 'HRV_SD2a', 'HRV_Cd', 'HRV_Ca', 'HRV_SDNNd', 'HRV_SDNNa',
        'HRV_DFA_alpha1', 'HRV_DFA_alpha1_ExpRange', 'HRV_DFA_alpha1_ExpMean', 'HRV_DFA_alpha1_DimRange',
        'HRV_DFA_alpha1_DimMean', 'HRV_ApEn', 'HRV_SampEn', 'gender', 'age', 'target_emotion', 'valence', 
        'arousal', 'dominance', 'stress_bin', 'participant'
    ])
    return data

In [22]:
def plot_settings():
    plt.rcParams['figure.figsize'] = [15, 9]
    plt.rcParams['font.size'] = 13

In [23]:
def processing(ecg_signal):
    ecg_cleaned = nk.ecg_clean(
        ecg_signal, sampling_rate=256, method="engzeemod2012")
    instant_peaks, rpeaks, = nk.ecg_peaks(ecg_cleaned, sampling_rate=256)
    rate = nk.ecg_rate(rpeaks, sampling_rate=256,
                       desired_length=len(ecg_cleaned))
    quality = nk.ecg_quality(ecg_cleaned, sampling_rate=256)

    signals = pd.DataFrame({"ECG_Raw": ecg_signal,
                            "ECG_Clean": ecg_cleaned,
                            "ECG_Rate": rate,
                            "ECG_Quality": quality})
    signals = pd.concat([signals, instant_peaks], axis=1)
    info = rpeaks

    return signals, info

In [24]:
def customiz_data_hrv(data):
    data = pd.DataFrame(data, columns=[
        'ECG_Rate_Mean', 'HRV_MeanNN', 'HRV_SDNN', 'HRV_RMSSD', 'HRV_SDSD', 'HRV_CVNN', 'HRV_CVSD',
        'HRV_MedianNN', 'HRV_MadNN', 'HRV_MCVNN', 'HRV_IQRNN', 'HRV_HTI', 'HRV_HF', 'HRV_HFn', 'HRV_LnHF',
        'HRV_SD1', 'HRV_SD2', 'HRV_SD1SD2', 'HRV_S', 'HRV_CSI', 'HRV_CVI', 'HRV_CSI_Modified', 'HRV_PIP',
        'HRV_IALS', 'HRV_PSS', 'HRV_GI', 'HRV_SI', 'HRV_AI', 'HRV_PI', 'HRV_C1d', 'HRV_C1a', 'HRV_SD1d', 
        'HRV_SD1a', 'HRV_C2d', 'HRV_C2a', 'HRV_SD2d', 'HRV_SD2a', 'HRV_Cd', 'HRV_Ca', 'HRV_SDNNd', 'HRV_SDNNa',
        'HRV_DFA_alpha1', 'HRV_DFA_alpha1_ExpRange', 'HRV_DFA_alpha1_ExpMean', 'HRV_DFA_alpha1_DimRange', 
        'HRV_DFA_alpha1_DimMean', 'HRV_ApEn', 'HRV_SampEn'
    ])
    return data

In [25]:
def PE_dreamer(path):
    raw = sio.loadmat(path)
    print("Dataset Load Done")
    print("  ")
    print("Processing and Extarctring Featuers....")
    df_ECG = feat_extract_ECG(raw)
    print("Processing and Extarctring Featuers Done")
    df_features = pd.concat([df_ECG], axis=1)
    df_participant_affective = participant_affective(raw)
    df_participant_affective["valence"] = (df_participant_affective
                                           ["valence"].astype(int))
    df_participant_affective["arousal"] = (df_participant_affective
                                           ["arousal"].astype(int))
    df_participant_affective["dominance"] = (df_participant_affective
                                             ["dominance"].astype(int))
    df = pd.concat([df_features, df_participant_affective], axis=1)

    data = df.loc[(df['target_emotion'] == 'calmness') |
                  (df['target_emotion'] == 'happiness') |
                  (df['target_emotion'] == 'sadness') |
                  (df['target_emotion'] == 'anger') |
                  (df['target_emotion'] == 'fear') |
                  (df['target_emotion'] == 'surprise')
                  ].copy()

    data['stress_bin'] = data['target_emotion'].map(
        {'calmness': 0, 'happiness': 1, 'sadness': 2,
            'anger': 3, 'fear': 4, 'surprise': 5}
    )

    data = customiz_data_ML(data)
    data.to_csv('Data/dataML_Modified.csv')

In [26]:
def PF_generated_ecg(heartRate):
    # PFecg.plot_settings()
    ecg = nk.ecg_simulate(sampling_rate=256, heart_rate=heartRate)
    # nk.signal_plot(ecg)
    signal, info = processing(ecg)
    #nk.ecg_plot(signal[:3000], sampling_rate=256)
    data = nk.ecg_intervalrelated(signal)
    data = customiz_data_hrv(data)
    data.to_csv("Data/HeartManual/generated_ECG_256hz.csv")
    return "Data/HeartManual/generated_ECG_256hz.csv"

In [27]:
def PF_ecg(path):
    # PFecg.plot_settings()
    ecg = pd.read_csv(path)
    # nk.signal_plot(ecg['ECG'])
    signal, info = nk.ecg_process(ecg["ECG"], sampling_rate=100)
    #nk.ecg_plot(signal[:3000], sampling_rate=100)
    data = nk.ecg_intervalrelated(signal)
    data = customiz_data_hrv(data)
    data.to_csv("Data/HeartManual/PF_ECG_100hz.csv")
    return "Data/HeartManual/PF_ECG_100hz.csv"

In [28]:
def preparing_ML_data(path):
    data = pd.read_csv(path)
    group_kfold = GroupKFold(n_splits=14)
    X = np.array(data.loc[:, 'ECG_Rate_Mean':'HRV_SampEn'])
    y = np.array(data['stress_bin'])
    groups = np.array(data['participant'])
    
    for train, test in group_kfold.split(X, y, groups):
        X_train, X_test = X[train], X[test]
        y_train, y_test = y[train], y[test]

    return X, y, groups, X_train, X_test, y_train, y_test

In [29]:
def run_clf(clf, X, y, groups, X_test, y_test):
    cv = GroupKFold(n_splits=14)
    score = []
    runtime = []
    for fold, (train, test) in enumerate(cv.split(X, y, groups)):
        clf.fit(X[train], y[train])
        start = time.time()
        score.append(clf.score(X_test, y_test))
        runtime.append(time.time() - start)
    return score, runtime

In [30]:
def supervised_model(train_data):
    
    X, y, groups, X_train, X_test, y_train, y_test = preparing_ML_data(
        train_data)
    
    results = []

    model = make_pipeline(MinMaxScaler(), SVC(gamma=2, C=1))

    score, runtime = run_clf(model, X, y, groups, X_test, y_test)

    results.append(["SVC", round(np.mean(score)*100, 1),
                   round(np.mean(runtime), 9)])
    result = pd.DataFrame(results, columns=['Name', 'Score', 'Runtime'])
    
    return model, result

In [31]:
def App(train_path: str, dreamer_path: str):
    
    if(not exists(train_path)):
        print("Trainning Data not found, Will generate new data")
        if(exists(dreamer_path)):
            PE_dreamer(dreamer_path)
        else:
            return "DREAMER Dataset Not Found"

    model_saved, result = supervised_model(train_path)
    joblib.dump(model_saved, 'model.pkl')
    return result

In [36]:
print(App("Data/dataML_Modified.csv","../Datasets/DREAMER.mat"))

  Name  Score   Runtime
0  SVC   93.5  0.001062


In [33]:
def run(path: str):
    emotions = ['calm', 'happy', 'sad', 'angry', 'fear', 'suprise']
    model = joblib.load('model.pkl')
    test = pd.read_csv(path)
    tester = np.array(test.loc[:, 'ECG_Rate_Mean':'HRV_SampEn'])
    emotion = model.predict(tester)
    return emotions[emotion[0]]

In [34]:
print(run('data/HeartEmotions/angry.csv'))

angry
